In [1]:
from ogb.nodeproppred import NodePropPredDataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch
from ogb.nodeproppred import Evaluator
import pandas as pd
import math
from tqdm import tqdm
from datasets import Dataset, DatasetDict
import numpy as np
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    AutoConfig,
    DataCollatorWithPadding,
    get_scheduler,
    get_linear_schedule_with_warmup
)
import evaluate

In [2]:
DATASET = "ogbn-products"
DATASET_ROOT = "/nlp/scr/ananjan/graph_datasets/"
OUTPUT_ROOT = "/nlp/scr/ananjan/graph_models/roberta_products/"
MODEL = "roberta-base"
BATCH_SIZE = 16
NUM_EPOCHS = 2
LR = 5e-5
WARMUP = 100
LOG_STEPS = 100
MAX_LEN = 512

In [3]:
# Load Dataset

dataset = NodePropPredDataset(name = DATASET, root = DATASET_ROOT)

In [4]:
! ls /nlp/scr/ananjan/graph_datasets/ogbn_products/mapping

Amazon-3M.raw	   labelidx2productcategory.csv  README.md
Amazon-3M.raw.zip  nodeidx2asin.csv


In [5]:
# Get Splits

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

In [6]:
# Get Labels for Node Classification

graph, label = dataset[0]
len(label)

2449029

In [7]:
# Load Text Label Mappings

labelidx2productcategeory = pd.read_csv('/nlp/scr/ananjan/graph_datasets/ogbn_products/mapping/labelidx2productcategory.csv')
print(len(labelidx2productcategeory))
labelidx2productcategeory.head()

47


,label idx,product category
0,0,Home & Kitchen
1,1,Health & Personal Care
2,2,Beauty
3,3,Sports & Outdoors
4,4,Books


In [8]:
# Load ASINs

node2asin = pd.read_csv('/nlp/scr/ananjan/graph_datasets/ogbn_products/mapping/nodeidx2asin.csv')
print(len(node2asin))
node2asin.head()

2449029


,node idx,asin
0,0,B00902X3L2
1,1,B000FW4BGM
2,2,B001NTYWFQ
3,3,B003DTLNVA
4,4,B00KFTCE28


In [9]:
# Load Paper Mappings
import json

txt_mapping = {}

with open('/nlp/scr/ananjan/graph_datasets/ogbn_products/mapping/Amazon-3M.raw/trn.json', 'r') as f:
    for line in f:
        jsonified = json.loads(line)
        txt_mapping[jsonified["uid"]] = jsonified["title"].strip()
        
with open('/nlp/scr/ananjan/graph_datasets/ogbn_products/mapping/Amazon-3M.raw/tst.json', 'r') as f:
    for line in f:
        jsonified = json.loads(line)
        txt_mapping[jsonified["uid"]] = jsonified["title"].strip()

In [10]:
# Dataset Creation

dataset_dict = {'text': [], 'labels': []}

for idx, l in tqdm(enumerate(label)):
    dataset_dict['labels'].append(l[0])
    asin = node2asin.iloc[idx]['asin']
    dataset_dict['text'].append(txt_mapping[asin])

dataset_dict['text'] = np.array(dataset_dict['text'])
dataset_dict['labels'] = np.array(dataset_dict['labels'])

2449029it [01:08, 35697.79it/s]


In [11]:
train_dataset, valid_dataset, test_dataset = {}, {}, {}

train_dataset['text'] = dataset_dict['text'][train_idx]
train_dataset['labels'] = dataset_dict['labels'][train_idx]
train_dataset = Dataset.from_dict(train_dataset)

valid_dataset['text'] = dataset_dict['text'][valid_idx]
valid_dataset['labels'] = dataset_dict['labels'][valid_idx]
valid_dataset = Dataset.from_dict(valid_dataset)

test_dataset['text'] = dataset_dict['text'][test_idx]
test_dataset['labels'] = dataset_dict['labels'][test_idx]
test_dataset = Dataset.from_dict(test_dataset)

fin_dataset = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset,
    'test': test_dataset
})

In [12]:
# train_dataset, valid_dataset, test_dataset = {}, {}, {}

# train_dataset['text'] = dataset_dict['text'][:100]
# train_dataset['label'] = dataset_dict['label'][:100]
# train_dataset = Dataset.from_dict(train_dataset)

# valid_dataset['text'] = dataset_dict['text'][:100]
# valid_dataset['label'] = dataset_dict['label'][:100]
# valid_dataset = Dataset.from_dict(valid_dataset)

# test_dataset['text'] = dataset_dict['text'][:100]
# test_dataset['label'] = dataset_dict['label'][:100]
# test_dataset = Dataset.from_dict(test_dataset)

# fin_dataset = DatasetDict({
#     'train': train_dataset,
#     'valid': valid_dataset,
#     'test': test_dataset
# })

# train_dataset[1]

In [13]:
# Init tokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

def tokenizer_helper(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

In [14]:
# Tokenize dataset

for split in fin_dataset:
    dataset = fin_dataset[split]
    dataset = dataset.map(tokenizer_helper, batched=True, batch_size=BATCH_SIZE)
    dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
    fin_dataset[split] = dataset

Map:   0%|          | 0/196615 [00:00<?, ? examples/s]

Map:   0%|          | 0/39323 [00:00<?, ? examples/s]

Map:   0%|          | 0/2213091 [00:00<?, ? examples/s]

In [15]:
# Compute Metrics for HF

metric = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    return metric.compute(predictions=np.argmax(logits, axis=-1), references=labels)

In [16]:
# Init model

model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=len(labelidx2productcategeory))

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# # HF Init and Training

# training_args = TrainingArguments(
#     output_dir=OUTPUT_ROOT,
#     num_train_epochs=NUM_EPOCHS,
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=BATCH_SIZE,
#     evaluation_strategy="epoch",
#     logging_dir=f"{OUTPUT_ROOT}/logs",
#     logging_steps=LOG_STEPS,
#     learning_rate=LR,
#     warmup_steps=WARMUP,
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     save_total_limit=1
# )

# trainer = Trainer(
#     model = model,
#     args = training_args,
#     train_dataset = fin_dataset['train'],
#     eval_dataset = fin_dataset['test'],
#     compute_metrics=compute_metrics
# )

# trainer.train()

In [ ]:
# tokenizer.save_pretrained(OUTPUT_ROOT + "checkpoint-2842/")

In [19]:
# PyTorch Init

train_dataloader = DataLoader(fin_dataset['train'], shuffle=True, batch_size=BATCH_SIZE)
eval_dataloader = DataLoader(fin_dataset['test'], batch_size=BATCH_SIZE)
optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = NUM_EPOCHS * len(train_dataloader)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer, num_warmup_steps=WARMUP, num_training_steps=num_training_steps
)

In [20]:
# PyTorch Train (required because HF logging is broken)
progress_bar = tqdm(range(num_training_steps))
device = torch.device("cuda")
model.to(device)

for epoch in range(NUM_EPOCHS):
    num_steps = 0
#     model.train()
#     for batch in train_dataloader:
#         batch_gpu = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch_gpu)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)
#         num_steps += 1
        
#         if (num_steps%LOG_STEPS == 0):
#             print(f'Step {num_steps}')
#             print(f'Train Loss {loss}')
    
#     print(f'Epoch {epoch} done')
    model.eval()
    
    metric = evaluate.load("accuracy")
    for batch in tqdm(eval_dataloader):
        batch_gpu = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch_gpu)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    print(f'Test Accuracy {metric.compute()}')
    
    metric = evaluate.load("accuracy")
    for batch in train_dataloader:
        batch_gpu = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch_gpu)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
    print(f'Train Accuracy {metric.compute()}')

  0%|▍                                                                                                    | 100/24578 [00:28<1:32:31,  4.41it/s]

Step 100
Train Loss 1.6136910915374756


  1%|▊                                                                                                    | 200/24578 [00:51<1:34:29,  4.30it/s]

Step 200
Train Loss 1.442386507987976


  1%|█▏                                                                                                   | 300/24578 [01:14<1:34:58,  4.26it/s]

...

 50%|██████████████████████████████████████████████████▏                                                  | 12200/24578 [48:51<49:30,  4.17it/s]

Step 12200
Train Loss 0.6607776284217834


 50%|██████████████████████████████████████████████████▌                                                  | 12289/24578 [49:12<41:49,  4.90it/s]

Epoch 0 done


KeyboardInterrupt: 

In [19]:
# Dump Logits
import pickle

OUT_MODEL = '/nlp/scr/ananjan/graph_models/roberta_all/checkpoint-2842'
device = torch.device("cuda")
model = AutoModelForSequenceClassification.from_pretrained(OUT_MODEL, num_labels=len(labelidx2arxivcategeory))
dump_dataset = Dataset.from_dict(dataset_dict)
dump_dataset = dump_dataset.map(tokenizer_helper, batched=True, batch_size=BATCH_SIZE)
dump_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

dataloader = DataLoader(dump_dataset, shuffle=False, batch_size=BATCH_SIZE)

Map:   0%|          | 0/169343 [00:00<?, ? examples/s]

In [22]:
from tqdm import tqdm

total_logits = []
model.to(device)
for batch in tqdm(dataloader):
    batch_gpu = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch_gpu)
        logits = list(outputs.logits.cpu().numpy())
        total_logits += logits
total_logits = np.array(total_logits)
        
with open("/nlp/scr/ananjan/graph_embeddings/finetuned/roberta_logits_arxiv.pkl", 'wb') as f:
    pickle.dump(total_logits, f)

100%|███████████████████████████████████████████████████████████████████| 10584/10584 [27:16<00:00,  6.47it/s]
